In [1]:
import scanpy as sc
#adata = sc.read_h5ad("../datasets/competition_support_set/k562_gwps.h5")
adata = sc.read_h5ad("/home/b5cc/sanjukta.b5cc/aracneseq/datasets/k562_5k.h5ad")
#/home/b5cc/sanjukta.b5cc/st3/datasets/k562_5k_with_celltype.h5ad
print(adata)
print(len(adata))

AnnData object with n_obs × n_vars = 5000 × 8563
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count'
    var: 'gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano'
5000


In [2]:
print(adata.X[100])

[0. 0. 0. ... 0. 3. 0.]


In [5]:
# Inspect gene_name (var) and gene (obs) columns
print("=== gene_name (var) ===")
if 'gene_name' in adata.var.columns:
    print(f"Total unique gene_name: {adata.var['gene_name'].nunique()}")
    print(adata.var['gene_name'].head())
else:
    print("gene_name column not found in adata.var")

print("\n=== gene (obs) ===")
if 'gene' in adata.obs.columns:
    print(f"Total unique gene: {adata.obs['gene'].nunique()}")
    print(adata.obs['gene'].head())
else:
    print("gene column not found in adata.obs")


=== gene_name (var) ===
Total unique gene_name: 8561
gene_id
ENSG00000237491    LINC01409
ENSG00000228794    LINC01128
ENSG00000188976        NOC2L
ENSG00000187961       KLHL17
ENSG00000188290         HES4
Name: gene_name, dtype: category
Categories (8561, object): ['A1BG', 'AAAS', 'AACS', 'AAGAB', ..., 'ZYG11A', 'ZYG11B', 'ZYX', 'ZZEF1']

=== gene (obs) ===
Total unique gene: 1651
cell_barcode
CATTGAGAGGCCGCTT-37     POLR3F
TGCTCCAAGACATCCT-11      BCAS2
CTCCCTCGTTCAGGTT-20     ZBTB17
GTAGAGGTCTATGCCC-37       PEF1
GTTGTCCGTTTCCAAG-34    TMEM127
Name: gene, dtype: category
Categories (1651, object): ['AAAS', 'AAMP', 'AARS', 'AARS2', ..., 'ZNHIT6', 'ZNRD1', 'ZRSR2', 'non-targeting']


In [3]:
#print(adata.obsm['X_hvg'][0][-1]) 
#for when its a hvg file 

from scripts.utils import plot_cellstate

example_state = adata.X[100]
print(adata.X[199])
print(max(example_state))
print(min(example_state))

plot_cellstate(example_state)

[0. 0. 0. ... 0. 0. 0.]
180.0
0.0
Saved cell state plot to: /home/b5cc/sanjukta.b5cc/st3/assets/cellstate_plot.png
  Total transcripts: 8563
  Expressed (>0): 2572
  Max expression: 180


'/home/b5cc/sanjukta.b5cc/st3/assets/cellstate_plot.png'

In [4]:
from scripts.utils import calc_sparsity 
calc_sparsity(example_state)

0.6996379773443886


In [6]:
# Count cells per gene in obs['gene']
if 'gene' in adata.obs.columns:
    gene_counts = adata.obs['gene'].value_counts()
    print("=== Cells per gene (obs['gene']) ===")
    print(f"Total genes with counts: {len(gene_counts)}")
    print("Top 10 genes by cell count:")
    print(gene_counts.head(10))
else:
    print("gene column not found in adata.obs")


=== Cells per gene (obs['gene']) ===
Total genes with counts: 1651
Top 10 genes by cell count:
gene
non-targeting    152
RPL3              31
NCBP2             23
KIF11             17
ARGLU1            14
PMPCB             14
SLC39A9           13
RPL34             12
NUP205            12
FBXO5             12
Name: count, dtype: int64


In [ ]:
# Inspect gene_name (var) and gene (obs) columns
print("=== gene_name (var) ===")
if 'gene_name' in adata.var.columns:
    print(f"Total unique gene_name: {adata.var['gene_name'].nunique()}")
    print(adata.var['gene_name'].head())
else:
    print("gene_name column not found in adata.var")

print("\n=== gene (obs) ===")
if 'gene' in adata.obs.columns:
    print(f"Total unique gene: {adata.obs['gene'].nunique()}")
    print(adata.obs['gene'].head())
else:
    print("gene column not found in adata.obs")
